In [0]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize #transform json data into pd df
import json

from geopy.geocoders import Nominatim
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from sklearn.cluster import KMeans

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/yesdeepakmittal/Coursera_Capstone/master/cordinate_data_toronto_wiki.csv',index_col = 0)
df.head()

,Postal Code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.754326,-79.449117
1,M4A,North York,Victoria Village,43.754326,-79.449117
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654174,-79.380812
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.754326,-79.449117
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.654174,-79.380812


In [5]:
print('Dataframe has {} Borough and {} neighborhood.'.format(len(df.Borough.unique()),df.shape[0]))

Dataframe has 10 Borough and 103 neighborhood.


Using geopy library to visualize the place after getting cordinates

In [6]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [50]:
LIMIT = 100

CLIENT_ID = 'PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H'
CLIENT_SECRET = '----'  # the client secret key was removed after its usage because it is secret key
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H
CLIENT_SECRET:----


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude'],
                                  )

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [23]:
toronto_venues.shape

(4781, 7)

In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,49,49,49,49,49,49
Alderwood / Long Branch,5,5,5,5,5,5
Bathurst Manor / Wilson Heights / Downsview North,3,3,3,3,3,3
Bayview Village,3,3,3,3,3,3
Bedford Park / Lawrence Manor East,3,3,3,3,3,3
...,...,...,...,...,...,...
Willowdale / Newtonbrook,3,3,3,3,3,3
Woburn,49,49,49,49,49,49
Woodbine Heights,13,13,13,13,13,13


In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 119 uniques categories.


In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bank,Bar,Beach,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,...,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Opera House,Optical Shop,Other Great Outdoors,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Steakhouse,Supermarket,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
num_top_venues = 5
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0           Coffee Shop  0.08
1  Fast Food Restaurant  0.06
2                 Hotel  0.06
3                 Beach  0.06
4        Clothing Store  0.06


----Alderwood / Long Branch----
         venue  freq
0  Supermarket   0.2
1       Garden   0.2
2   Playground   0.2
3  Coffee Shop   0.2
4     Pharmacy   0.2


----Bathurst Manor / Wilson Heights / Downsview North----
                        venue  freq
0   Middle Eastern Restaurant  0.33
1                 Coffee Shop  0.33
2    Mediterranean Restaurant  0.33
3  Modern European Restaurant  0.00
4       Performing Arts Venue  0.00


----Bayview Village----
                        venue  freq
0   Middle Eastern Restaurant  0.33
1                 Coffee Shop  0.33
2    Mediterranean Restaurant  0.33
3  Modern European Restaurant  0.00
4       Performing Arts Venue  0.00


----Bedford Park / Lawrence Manor East----
                        venue  freq
0   Middle Eastern Restaurant  0.33
1      

####Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.


In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore
1,Alderwood / Long Branch,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
3,Bayview Village,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
4,Bedford Park / Lawrence Manor East,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop


##Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 1, 1, 1, 3, 0, 3, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.754326,-79.449117,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
1,M4A,North York,Victoria Village,43.754326,-79.449117,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654174,-79.380812,3,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Café,Diner,Middle Eastern Restaurant,Breakfast Spot,Bar
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.754326,-79.449117,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.654174,-79.380812,3,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Café,Diner,Middle Eastern Restaurant,Breakfast Spot,Bar


In [44]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Cluster 1

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,0,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore
12,Scarborough,0,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore
18,Scarborough,0,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore
22,Scarborough,0,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore
26,Scarborough,0,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore
32,Scarborough,0,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore
38,Scarborough,0,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore
44,Scarborough,0,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore
51,Scarborough,0,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore
58,Scarborough,0,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Beach,Hotel,Seafood Restaurant,Department Store,Arcade,Bookstore


Cluster 2

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
1,North York,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
3,North York,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
7,North York,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
10,North York,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
13,North York,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
27,North York,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
28,North York,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
33,North York,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
34,North York,1,Coffee Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Women's Store,Comic Shop,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop


Cluster 3

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,2,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar
11,Etobicoke,2,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar
17,Etobicoke,2,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar
70,Etobicoke,2,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar
77,Etobicoke,2,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar
88,Etobicoke,2,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar
89,Etobicoke,2,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar
93,Etobicoke,2,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar
94,Etobicoke,2,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar
98,Etobicoke,2,Coffee Shop,Garden,Pharmacy,Playground,Supermarket,Garden Center,Gastropub,General Travel,Comic Shop,Gay Bar


Cluster 4

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Café,Diner,Middle Eastern Restaurant,Breakfast Spot,Bar
4,Downtown Toronto,3,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Café,Diner,Middle Eastern Restaurant,Breakfast Spot,Bar
9,Downtown Toronto,3,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Café,Diner,Middle Eastern Restaurant,Breakfast Spot,Bar
15,Downtown Toronto,3,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Café,Diner,Middle Eastern Restaurant,Breakfast Spot,Bar
16,York,3,Pub,Café,Bar,Coffee Shop,Plaza,Historic Site,French Restaurant,Cocktail Bar,Burger Joint,Italian Restaurant
19,East Toronto,3,Clothing Store,Coffee Shop,Restaurant,Theater,Plaza,Café,Seafood Restaurant,Cosmetics Shop,Thai Restaurant,Diner
20,Downtown Toronto,3,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Café,Diner,Middle Eastern Restaurant,Breakfast Spot,Bar
21,York,3,Pub,Café,Bar,Coffee Shop,Plaza,Historic Site,French Restaurant,Cocktail Bar,Burger Joint,Italian Restaurant
24,Downtown Toronto,3,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Café,Diner,Middle Eastern Restaurant,Breakfast Spot,Bar
25,Downtown Toronto,3,Coffee Shop,Clothing Store,Hotel,Restaurant,Sandwich Place,Café,Diner,Middle Eastern Restaurant,Breakfast Spot,Bar


Cluster 5

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,4,Thrift / Vintage Store,Burger Joint,Sushi Restaurant,American Restaurant,Optical Shop,Music Store,Seafood Restaurant,Donut Shop,Hobby Shop,Bar
14,East York,4,Thrift / Vintage Store,Burger Joint,Sushi Restaurant,American Restaurant,Optical Shop,Music Store,Seafood Restaurant,Donut Shop,Hobby Shop,Bar
23,East York,4,Thrift / Vintage Store,Burger Joint,Sushi Restaurant,American Restaurant,Optical Shop,Music Store,Seafood Restaurant,Donut Shop,Hobby Shop,Bar
29,East York,4,Thrift / Vintage Store,Burger Joint,Sushi Restaurant,American Restaurant,Optical Shop,Music Store,Seafood Restaurant,Donut Shop,Hobby Shop,Bar
35,East York,4,Thrift / Vintage Store,Burger Joint,Sushi Restaurant,American Restaurant,Optical Shop,Music Store,Seafood Restaurant,Donut Shop,Hobby Shop,Bar
